In [ ]:
#!pip install seaborn
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import warnings
import gzip, pickle
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')
from IPython.display import Image, display
!pip install seaborn -qq


In [ ]:
display(Image(filename='a.jpg', width=800,height=500))  

In [ ]:
DATA_PATH ="mnist.pkl.gz"
with gzip.open(DATA_PATH, 'rb') as f:
    (X_train, y_train), (X_valid, y_valid), (X_test,  y_test) = pickle.load(f, encoding='latin1')

# As a sanity check, we print out the size of the data.
print('Training data shape:    ', X_train.shape)
print('Training labels shape:  ', y_train.shape)
print('Validation data shape:  ', X_valid.shape)
print('Validation labels shape:', y_valid.shape)
print('Test data shape:        ', X_test.shape)
print('Test labels shape:      ', y_test.shape)

In [ ]:
# compute mean vector from training data
mu = np.mean(X_train, axis=0)

# remove mean vector from all data
X_train -= mu
X_valid -= mu
X_test  -= mu

Plotting the mean vector

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(mu.reshape(28, 28), interpolation='nearest', cmap=plt.cm.Greys)
plt.xticks([])
plt.yticks([])
plt.title("Mean value of features")
plt.show()

## Multi-Class Logistic Regression

### Two-class logistic regression

$$h_\theta(x) = g(\theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n) = g(\theta^T x + \theta_0) = \frac{1}{1 + e^{-(\theta^T x + \theta_0)}}$$

Notice that here, we have not included $x_0 = 1$ and hence treated the bias separetly.


### Multi-class logistic regression

<img src='multi-class-lr-vectorized.png' width='75%'>

$$scores = X \Theta + \theta_0$$
where:
- $X \in \mathbb{R}^{m \times n}$
- $\Theta \in \mathbb{R}^{n \times c}$
- $\theta_0 \in \mathbb{R}^{1 \times c}$
- $scores \in \mathbb{R}^{m \times c}$

In python, we can compute the scores using the following statement:
```python
    scores = X @ Theta + theta0
```


#### Prediction

After computing scores, we have `c` predictions for each input data. By taking max over these predictions, we can easily compute the class label for each input data:
```python
    y_pred = np.argmax(scores, axis=1)
```

In [ ]:
def predict(W, b, X):
    scores = X @ W + b
    return np.argmax(scores, axis=1)

#### Prediction for random parameters

In [ ]:
def accuracy(y_pred, y_true):
    return 100. * np.mean(y_pred == y_true)

In [ ]:
c = 10                # number of classes 
n = X_train.shape[1]  # number of features

# init parameters randomly
W = 0.01 * np.random.randn(n, c)
b = np.zeros(c)

# predict classes and compute accuracy
y_pred = predict(W, b, X_train)
acc = accuracy(y_pred, y_train)  # this function is defined in utils.py
print("Accuracy = {:.2f}%".format(acc))

## Softmax classifier

<img src='softmax_classifier.png' width='50%'>

$$probs = \text{softmax}(scores)$$

### Softmax function

$$\text{softmax}(x^{(i)})_k = \frac{e^{s_k}}{\sum_{j=1}^{c} e^{s_j}}$$

## Training Softmax Classifier

**Training data:**
- $\{(x^{(i)}, y^{(i)})\}_{i=1}^{m}$,
- $x^{(i)} \in \mathbb{R}^{n}$,
- $y \in \{1, 2, \dots, c\}$
    

**Parameters:**
- weights: $W \in \mathbb{R}^{n \times c}$
- biases: $b \in \mathbb{R}^{c}$

**Loss function:** softmax loss function + regularization
$$L(W, b) = \frac{1}{m} \sum_{i=1}^{m} -\log(p_{y^{(i)}}) + \frac{\lambda}{2} \lVert W \rVert_{2}^{2}$$

As usual, we need to find the optimal $W$ and $b$ by minimizing $L(W, b)$. Again, we can use **gradient descent** algorithm or any other optimization method to minimize this loss function.

#### Gradient

$$\frac{\partial L_i}{\partial w^{(k)}} = \frac{\partial L_i}{\partial s_k} \frac{\partial s_k}{\partial w^{(k)}} = \frac{\partial L_i}{\partial s_k} x^{(i)}$$

for $k = y^{(i)}$:
$$\frac{\partial L_i}{\partial s_k} = p_k - 1$$

for $k \neq y^{(i)}$:
$$\frac{\partial L_i}{\partial s_k} = p_k$$

In [ ]:

def softmax(x):
    x = x - np.max(x, axis=1, keepdims=True)
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [ ]:
def softmax_loss(W, b, X_batch, y_batch, mode='train'):
    bs = X_batch.shape[0]  # batch size
    
    scores = X_batch @ W + b
    probs = softmax(scores)
    loss = -np.sum(np.log(probs[range(bs), y_batch])) / bs
    
    if mode == 'test':
        return loss
    
    # compute gradients w.r.t scores
    dscores = np.copy(probs)
    dscores[range(bs), y_batch] -= 1.0
    dscores /= bs
    
    # compute gradients w.r.t W and b
    db = dscores.sum(axis=0)
    dW = X_batch.T @ dscores
    
    return loss, dW, db

In [ ]:
display(Image(filename='c.png', width=500)) 

In [ ]:
def mini_batch_gradient_descent(X_train, y_train, X_valid, y_valid, batch_size=32, 
                                alpha=0.01, lmbda=1e-4, num_epochs=100):
    
    m, n = X_train.shape
    num_batches = m % batch_size
    
    report = "Epoch {:3d}: training loss = {:.2f} | validation loss = {:.2f}"
    
    # init parameters randomly
    W = np.random.randn(n, 10) * 0.001
    b = np.zeros((10,))
    
    for epoch in range(num_epochs):
        train_loss = 0.
        
        for batch in range(num_batches):
            
            # select a random mini-batch
            idx = np.random.choice(m, batch_size, replace=False)
            X_batch, y_batch = X_train[idx], y_train[idx]
            
            # compute loss and gradient
            loss, dW, db = softmax_loss(W, b, X_batch, y_batch)  # data loss
            loss += 0.5 * lmbda * np.sum(W ** 2)                 # regularization loss
            dW += lmbda * W
            
            train_loss += loss
            
            # update parameters            
            b = b - alpha * db
            W = W - alpha * dW
        
        # report stats after each epoch
        train_loss /= num_batches        
        valid_loss = softmax_loss(W, b, X_valid, y_valid, mode='test')
        print(report.format(epoch+1, train_loss, valid_loss))
    
    return W, b

In [ ]:
# hyper-parameters
alpha = 1e-2
lmbda = 1e-4
batch_size = 64
num_epochs = 10

# run mini-batch gradient descent
W, b = mini_batch_gradient_descent(X_train, y_train, X_valid, y_valid, 
                                   batch_size=batch_size, alpha=alpha,
                                   lmbda=lmbda, num_epochs=num_epochs)

In [ ]:
train_acc = accuracy(predict(W, b, X_train), y_train)
valid_acc = accuracy(predict(W, b, X_valid), y_valid)

print('Training accuracy =   {:.2f}%'.format(train_acc))
print('Validation accuracy = {:.2f}%'.format(valid_acc))

#### Prediction for test data

In [ ]:
test_acc = accuracy(predict(W, b, X_test), y_test)
print('Test accuracy =   {:.2f}%'.format(train_acc))

### Visualizing weights

In [ ]:
plt.figure(figsize=(10, 4))

for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(W[:, i].reshape((28, 28)), cmap=plt.cm.coolwarm)
    plt.xticks([])
    plt.yticks([])
    plt.title("%d" % i)
plt.show()


Non-linear Classification: Neural Networks

In [ ]:

def Softmax_loss(scores, y, mode='train'):
    m = scores.shape[0]
    probs = softmax(scores)
    loss = -np.sum(np.log(probs[range(m), y])) / m
    
    if mode != 'train':
        return loss
    
    # backward
    dscores = probs
    dscores[range(m), y] -= 1.0
    dscores /= m
    
    return loss, dscores

In [ ]:
display(Image(filename='b.png', width=500)) 

In [ ]:

class TwoLayerNeuralNetwork:
    
    def __init__(self, num_features=784, num_hiddens=20, num_classes=10):
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        
        # random initialization: create random weights, set all biases to zero
        self.params = {}
        self.params['W1'] = np.random.randn(num_features, num_hiddens) * 0.001
        self.params['W2'] = np.random.randn(num_hiddens,  num_classes) * 0.001
        self.params['b1'] = np.zeros((num_hiddens,))
        self.params['b2'] = np.zeros((num_classes,))
        
    def forward(self, X):
        # forward step
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        
        # forward step
        h_in = X @ W1 + b1       # hidden layer input
        h = np.maximum(0, h_in)  # hidden layer output (using ReLU)
        scores = h @ W2 + b2     # neural net output
        
        return scores
                            
    def train_step(self, X, y):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        
        # forward step
        h_in = X @ W1 + b1       # hidden layer input
        h = np.maximum(0, h_in)  # hidden layer output (using ReLU)
        scores = h @ W2 + b2     # neural net output
        
        # compute loss
        loss, dscores = Softmax_loss(scores, y)
        
        # backward step
        db2 = dscores.sum(axis=0)
        dW2 = h.T @ dscores
        
        dh = dscores @ W2.T
        dh[h_in < 0] = 0.0
        db1 = dh.sum(axis=0)
        dW1 = X.T @ dh
        
        gradient = {'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2}
                
        return loss, gradient
        
    def train(self, X_train, y_train, X_valid, y_valid, batch_size=50, 
              alpha=0.001, lmbda=0.0001, num_epochs=10):
        
        m, n = X_train.shape        
        num_batches = m // batch_size
        
        report = "{:3d}: training loss = {:.2f} | validation loss = {:.2f}"
        
        losses = []
        for epoch in range(num_epochs):
            train_loss = 0.0
            
            for _ in range(num_batches):
                W1, b1 = self.params['W1'], self.params['b1']
                W2, b2 = self.params['W2'], self.params['b2']
                
                # select a random mini-batch
                batch_idx = np.random.choice(m, batch_size, replace=False)
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]

                # train on mini-batch
                data_loss, gradient = self.train_step(X_batch, y_batch)
                reg_loss = 0.5 * (np.sum(W1 ** 2) + np.sum(W2 ** 2))
                train_loss += (data_loss + lmbda * reg_loss)
                losses.append(data_loss + lmbda * reg_loss)

                # regularization
                gradient['W1'] += lmbda * W1
                gradient['W2'] += lmbda * W2

                # update parameters
                for p in self.params:
                    self.params[p] = self.params[p] - alpha * gradient[p]
            
            # report training loss and validation loss
            train_loss /= num_batches
            valid_loss = Softmax_loss(self.forward(X_valid), y_valid, mode='test')
            print(report.format(epoch + 1, train_loss, valid_loss))
        
        return losses
    
    def predict(self, X):
        """ Predict labels for input data.
        """
        scores = self.forward(X)
        return np.argmax(scores, axis=1)
    
    def predict_proba(self, X):
        """ Predict probabilties of classes for each input data.
        """
        scores = self.forward(X)
        return softmax(scores)

In [ ]:
mlp = TwoLayerNeuralNetwork(num_hiddens=20)
losses = mlp.train(X_train, y_train, X_valid, y_valid, 
                   alpha=0.05, lmbda=0.001, num_epochs=10)

In [ ]:
# Manually define 5 configurations
import matplotlib.pyplot as plt
configurations = [
    {'num_hiddens': 10, 'alpha': 0.01, 'lambda': 0.0001},
    {'num_hiddens': 20, 'alpha': 0.05, 'lambda': 0.001},
    {'num_hiddens': 30, 'alpha': 0.1, 'lambda': 0.01},
    {'num_hiddens': 40, 'alpha': 0.2, 'lambda': 0.1},
    {'num_hiddens': 50, 'alpha': 0.3, 'lambda': 1}
]
training_accuracies = []
test_accuracies=[]
lossess=[]

# Initialize a dictionary to store the losses for each configuration
config_losses = {}

# Loop through the defined configurations
for config in configurations:
    # Extract the parameters for the current configuration
    num_hiddens = config['num_hiddens']
    alpha = config['alpha']
    lambda_ = config['lambda']  # Using lambda_ to avoid conflict with the keyword lambda

    # Initialize the neural network with the current configuration
    mlp = TwoLayerNeuralNetwork(num_hiddens=num_hiddens)
    
    # Train the neural network and get the losses
    losses = mlp.train(X_train, y_train, X_valid, y_valid, alpha=alpha, lmbda=lambda_, num_epochs=10)
    lossess.append(losses)
    # Store the losses with the corresponding configuration
    config_losses[(num_hiddens, alpha, lambda_)] = losses
    #predictions = mlp.predict(X_train)
    train_acc = accuracy(mlp.predict(X_train), y_train)
    training_accuracies.append(train_acc)

    test_acc = accuracy(mlp.predict(X_test), y_test)
    test_accuracies.append(test_acc)
    

    # Print the configuration and its final loss for comparison
    print(f"Config: num_hiddens={num_hiddens}, alpha={alpha}, lambda={lambda_}, Final Loss: {losses[-1]}")

labels = [f"Config {i+1}" for i in range(len(configurations))]

# Create the bar chart
plt.figure(figsize=(10, 6))
plt.bar(labels, training_accuracies, color='skyblue')

# Add title and labels with more information
plt.title('Training Accuracies of Different Configurations')
plt.xlabel('Configuration')
plt.ylabel('Training Accuracy')

# Optional: Display the accuracy values on top of the bars
for i, acc in enumerate(training_accuracies):
    plt.text(i, acc, f"{acc:.2f}", ha='center', va='bottom')

plt.show() 

plt.figure(figsize=(10, 6))
plt.bar(labels, test_accuracies, color='skyblue')

# Add title and labels with more information
plt.title('Test Accuracies of Different Configurations')
plt.xlabel('Configuration')
plt.ylabel('Test Accuracy')

# Optional: Display the accuracy values on top of the bars
for i, acc in enumerate(test_accuracies):
    plt.text(i, acc, f"{acc:.2f}", ha='center', va='bottom')

plt.show()



In [ ]:
#here I plot loss function for each configuration seperatly
for i in range(5):
    print(i)
    plt.plot(lossess[i])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(range(0, 10001, 1000), range(0, 11))
    plt.show()

### Neural networks in scikit learn

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(20,), learning_rate='adaptive', alpha=1.0, max_iter=10, verbose=1)
model.fit(X_train, y_train);

In [ ]:
train_acc = model.score(X_train, y_train)
print("Train accuracy = {:.2f}%".format(train_acc * 100))

test_acc = model.score(X_test, y_test)
print("Test accuracy  = {:.2f}%".format(test_acc * 100))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming 'y_true' and 'y_pred' are defined
# Compute the confusion matrix
y_pre=model.predict(X_test)
cm = confusion_matrix(y_test, y_pre)

unique_labels = np.unique(y_test)

# Normalize the confusion matrix to get percentages
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plotting
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (Percentage)')
plt.show()
